# Análisis del Dataset Zoo - Clasificación Multiclase

Este notebook implementa y evalúa 6 algoritmos de clasificación en el dataset Zoo de UCI:
1. Naive Bayes Gaussiano
2. MLE Multivariante (Full Bayesian Gaussian)
3. Histogram Bayes
4. Parzen Windows
5. k-NN Density Bayes
6. k-NN Rule

Dataset: 17 atributos (15 binarios + 1 numérico + 1 clase), 7 clases de animales

## 1. Importación de librerías

In [102]:
import pandas as pd
import numpy as np
import os
import warnings
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KernelDensity
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from scipy.stats import multivariate_normal
from sklearn.model_selection import cross_val_score

# Silenciar warnings
os.environ['LOKY_MAX_CPU_COUNT'] = '4'
warnings.filterwarnings('ignore', category=UserWarning, module='joblib')

print("✓ Librerías importadas correctamente")

✓ Librerías importadas correctamente


## 2. Carga y análisis exploratorio del dataset

In [103]:
# Clases del Zoo dataset (1-7 -> labels para report)
class_names = ['mammal', 'bird', 'reptile', 'fish', 'amphibian', 'invertebrate', 'insect']

print("=" * 80)
print("ANÁLISIS DEL DATASET ZOO (Multiclass Classification)")
print("=" * 80)

# Cargar datos: zoo.data (col 0: animal name (ignorar), col 1-17: features, col 18: class 1-7)
df = pd.read_csv('./data_zoo/zoo_balanced.csv')
# df = pd.read_csv('./data_zoo/zoo.csv')
df.columns = ['animal'] + [f'feature_{i}' for i in range(1, 17)] + ['class']
X = df.iloc[:, 1:-1]  # Features 1-17
y = df.iloc[:, -1].values - 1  # Clase 0-6 para sklearn

print("\nInformación del dataset:")
print(f"Forma: {X.shape} (instancias x features)")
print(f"Clases: {len(np.unique(y))} (multiclass: {class_names})")
print("\nDistribución de clases:")
unique, counts = np.unique(y, return_counts=True)
for i, (cls, count) in enumerate(zip(class_names, counts)):
    print(f"Clase {i+1} ({cls}): {count} muestras ({count/len(y)*100:.1f}%)")

# Mostrar primeras filas
print("\nPrimeras 5 filas del dataset:")
print(df.head())

ANÁLISIS DEL DATASET ZOO (Multiclass Classification)

Información del dataset:
Forma: (287, 16) (instancias x features)
Clases: 7 (multiclass: ['mammal', 'bird', 'reptile', 'fish', 'amphibian', 'invertebrate', 'insect'])

Distribución de clases:
Clase 1 (mammal): 41 muestras (14.3%)
Clase 2 (bird): 41 muestras (14.3%)
Clase 3 (reptile): 41 muestras (14.3%)
Clase 4 (fish): 41 muestras (14.3%)
Clase 5 (amphibian): 41 muestras (14.3%)
Clase 6 (invertebrate): 41 muestras (14.3%)
Clase 7 (insect): 41 muestras (14.3%)

Primeras 5 filas del dataset:
     animal  feature_1  feature_2  feature_3  feature_4  feature_5  feature_6  \
0  aardvark          1          0          0          1          0          0   
1  antelope          1          0          0          1          0          0   
2      bear          1          0          0          1          0          0   
3      boar          1          0          0          1          0          0   
4   buffalo          1          0          0  

## 3. División del dataset (Train-Test) y configuración de validación cruzada

In [104]:
# División: 80% train - 20% test, estratificada para multiclass
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=41, stratify=y
)

print(f"Total de muestras: {len(X)}")
print(f"Train: {len(X_train)} ({len(X_train)/len(X)*100:.1f}%)")
print(f"Test: {len(X_test)} ({len(X_test)/len(X)*100:.1f}%)")

print("\nDistribución en train:")
print(pd.Series(y_train).value_counts().sort_index())
print("\nDistribución en test:")
print(pd.Series(y_test).value_counts().sort_index())

# CV estratificado (5 folds) para train
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=41)
print("\n✓ Configuración de validación cruzada: 5-fold estratificada")

Total de muestras: 287
Train: 229 (79.8%)
Test: 58 (20.2%)

Distribución en train:
0    32
1    33
2    33
3    32
4    33
5    33
6    33
Name: count, dtype: int64

Distribución en test:
0    9
1    8
2    8
3    9
4    8
5    8
6    8
Name: count, dtype: int64

✓ Configuración de validación cruzada: 5-fold estratificada


## 4. Función auxiliar para evaluación de modelos

In [105]:
# Función helper para evaluar modelo (pred en test + report)
def evaluate_model(model, X_test, y_test, model_name):
    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)
    f1_mac = f1_score(y_test, preds, average='macro')
    print(f"\n--- Resultados en Test ({model_name}) ---")
    print(f"Accuracy: {acc:.4f}")
    print(f"F1-macro: {f1_mac:.4f}")
    print("\nReporte de clasificación:")
    print(classification_report(y_test, preds, target_names=class_names))
    cm = confusion_matrix(y_test, preds)
    print("\nMatriz de confusión:")
    print(cm)
    return acc, f1_mac, cm

print("✓ Función de evaluación definida")

✓ Función de evaluación definida


## 5. Modelo 1: Naive Bayes Gaussiano

In [106]:
print("=" * 80)
print("1. NAIVE BAYES GAUSSIANO")
print("=" * 80)

nb = GaussianNB()
nb.fit(X_train, y_train)
nb_acc, nb_f1, nb_cm = evaluate_model(nb, X_test, y_test, "Naive Bayes")

# CV score para NB (sin hypers)
nb_cv_scores = cross_val_score(nb, X_train, y_train, cv=skf, scoring='f1_macro')
print(f"\nCV F1-macro (mean ± std): {nb_cv_scores.mean():.4f} ± {nb_cv_scores.std():.4f}")

1. NAIVE BAYES GAUSSIANO

--- Resultados en Test (Naive Bayes) ---
Accuracy: 0.8966
F1-macro: 0.8885

Reporte de clasificación:
              precision    recall  f1-score   support

      mammal       1.00      1.00      1.00         9
        bird       1.00      1.00      1.00         8
     reptile       1.00      0.50      0.67         8
        fish       1.00      1.00      1.00         9
   amphibian       0.64      0.88      0.74         8
invertebrate       0.89      1.00      0.94         8
      insect       0.88      0.88      0.88         8

    accuracy                           0.90        58
   macro avg       0.91      0.89      0.89        58
weighted avg       0.92      0.90      0.89        58


Matriz de confusión:
[[9 0 0 0 0 0 0]
 [0 8 0 0 0 0 0]
 [0 0 4 0 4 0 0]
 [0 0 0 9 0 0 0]
 [0 0 0 0 7 0 1]
 [0 0 0 0 0 8 0]
 [0 0 0 0 0 1 7]]

CV F1-macro (mean ± std): 0.8782 ± 0.0354


## 6. Modelo 2: MLE Multivariante (Full Bayesian Gaussian)

In [107]:
print("=" * 80)
print("2. MLE MULTIVARIANTE (Full Bayesian Gaussian)")
print("=" * 80)

class FullGaussianBayes:
    def __init__(self):
        self.priors = None
        self.means = None
        self.covs = None
        self.classes = None
    
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.priors = np.bincount(y) / len(y)
        self.means = np.array([X[y == c].mean(axis=0) for c in self.classes])
        self.covs = np.array([np.cov(X[y == c].T) + 1e-6 * np.eye(X.shape[1]) for c in self.classes])
        return self
    
    def predict(self, X):
        n_samples = X.shape[0]
        ll = np.zeros((n_samples, len(self.classes)))
        for i, c in enumerate(self.classes):
            ll[:, i] = multivariate_normal(mean=self.means[i], cov=self.covs[i]).logpdf(X)
        posteriors = np.exp(ll) * self.priors
        posteriors /= posteriors.sum(axis=1, keepdims=True)
        return np.argmax(posteriors, axis=1)

print("✓ Clase FullGaussianBayes definida")

2. MLE MULTIVARIANTE (Full Bayesian Gaussian)
✓ Clase FullGaussianBayes definida


In [108]:
mle = FullGaussianBayes()
mle.fit(X_train.values, y_train)
mle_acc, mle_f1, mle_cm = evaluate_model(mle, X_test.values, y_test, "MLE Full")

# CV para MLE (custom)
def cv_full_bayes(X_train, y_train, cv):
    scores = []
    for train_idx, val_idx in cv.split(X_train, y_train):
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]
        model = FullGaussianBayes()
        model.fit(X_tr.values, y_tr)
        preds = model.predict(X_val.values)
        scores.append(f1_score(y_val, preds, average='macro'))
    return np.array(scores)

mle_cv_scores = cv_full_bayes(pd.DataFrame(X_train), y_train, skf)
print(f"\nCV F1-macro (mean ± std): {mle_cv_scores.mean():.4f} ± {mle_cv_scores.std():.4f}")


--- Resultados en Test (MLE Full) ---
Accuracy: 0.9310
F1-macro: 0.9281

Reporte de clasificación:
              precision    recall  f1-score   support

      mammal       0.90      1.00      0.95         9
        bird       1.00      1.00      1.00         8
     reptile       1.00      0.88      0.93         8
        fish       1.00      1.00      1.00         9
   amphibian       0.88      0.88      0.88         8
invertebrate       0.89      1.00      0.94         8
      insect       0.86      0.75      0.80         8

    accuracy                           0.93        58
   macro avg       0.93      0.93      0.93        58
weighted avg       0.93      0.93      0.93        58


Matriz de confusión:
[[9 0 0 0 0 0 0]
 [0 8 0 0 0 0 0]
 [0 0 7 0 1 0 0]
 [0 0 0 9 0 0 0]
 [0 0 0 0 7 0 1]
 [0 0 0 0 0 8 0]
 [1 0 0 0 0 1 6]]

CV F1-macro (mean ± std): 0.8889 ± 0.0595


C:\Users\jordi\AppData\Local\Temp\ipykernel_8164\2355046959.py:25: RuntimeWarning: invalid value encountered in divide
  posteriors /= posteriors.sum(axis=1, keepdims=True)
C:\Users\jordi\AppData\Local\Temp\ipykernel_8164\2355046959.py:25: RuntimeWarning: invalid value encountered in divide
  posteriors /= posteriors.sum(axis=1, keepdims=True)
C:\Users\jordi\AppData\Local\Temp\ipykernel_8164\2355046959.py:25: RuntimeWarning: invalid value encountered in divide
  posteriors /= posteriors.sum(axis=1, keepdims=True)
C:\Users\jordi\AppData\Local\Temp\ipykernel_8164\2355046959.py:25: RuntimeWarning: invalid value encountered in divide
  posteriors /= posteriors.sum(axis=1, keepdims=True)
C:\Users\jordi\AppData\Local\Temp\ipykernel_8164\2355046959.py:25: RuntimeWarning: invalid value encountered in divide
  posteriors /= posteriors.sum(axis=1, keepdims=True)


## 7. Modelo 3: Histogram Bayes

In [109]:
print("=" * 80)
print("3. DENSIDAD NO PARAMÉTRICA - HISTOGRAMA")
print("=" * 80)

class HistogramBayes:
    def __init__(self, bins=2):
        self.bins = bins
        self.priors = None
        self.hist_per_class = None
        self.edges = None
    
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.priors = np.bincount(y) / len(y)
        self.hist_per_class = {}
        for c in self.classes:
            X_c = X[y == c]
            hists = []
            edges_list = []
            for feat in range(X.shape[1]):
                hist, edges = np.histogram(X_c.iloc[:, feat], bins=self.bins, density=True)
                hists.append(hist)
                edges_list.append(edges)
            self.hist_per_class[c] = (np.array(hists), edges_list)
        self.edges = edges_list[0] if edges_list else None
        return self
    
    def _density_hist(self, x, c):
        hists, edges = self.hist_per_class[c]
        dens = 1.0
        for i, feat_val in enumerate(x):
            bin_idx = np.digitize(feat_val, edges[i]) - 1
            if 0 <= bin_idx < len(hists[i]):
                dens *= hists[i][bin_idx]
            else:
                dens *= 0
        return dens
    
    def predict(self, X):
        n_samples = len(X)
        preds = np.zeros(n_samples, dtype=int)
        for i in range(n_samples):
            posteriors = []
            for c in self.classes:
                dens = self._density_hist(X.iloc[i], c)
                post = self.priors[c] * dens
                posteriors.append(post)
            preds[i] = self.classes[np.argmax(posteriors)]
        return preds

print("✓ Clase HistogramBayes definida")

3. DENSIDAD NO PARAMÉTRICA - HISTOGRAMA
✓ Clase HistogramBayes definida


In [110]:
hist_bayes = HistogramBayes(bins=2)
hist_bayes.fit(pd.DataFrame(X_train), y_train)
hist_acc, hist_f1, hist_cm = evaluate_model(hist_bayes, pd.DataFrame(X_test), y_test, "Histogram Bayes")

# CV para Histogram (custom)
def cv_hist_bayes(X_train, y_train, cv):
    scores = []
    for train_idx, val_idx in cv.split(X_train, y_train):
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]
        model = HistogramBayes()
        model.fit(X_tr, y_tr)
        preds = model.predict(X_val)
        scores.append(f1_score(y_val, preds, average='macro'))
    return np.array(scores)

hist_cv_scores = cv_hist_bayes(pd.DataFrame(X_train), y_train, skf)
print(f"\nCV F1-macro (mean ± std): {hist_cv_scores.mean():.4f} ± {hist_cv_scores.std():.4f}")


--- Resultados en Test (Histogram Bayes) ---
Accuracy: 0.1724
F1-macro: 0.0707

Reporte de clasificación:
              precision    recall  f1-score   support

      mammal       0.16      1.00      0.27         9
        bird       0.00      0.00      0.00         8
     reptile       0.00      0.00      0.00         8
        fish       0.00      0.00      0.00         9
   amphibian       0.00      0.00      0.00         8
invertebrate       1.00      0.12      0.22         8
      insect       0.00      0.00      0.00         8

    accuracy                           0.17        58
   macro avg       0.17      0.16      0.07        58
weighted avg       0.16      0.17      0.07        58


Matriz de confusión:
[[9 0 0 0 0 0 0]
 [8 0 0 0 0 0 0]
 [8 0 0 0 0 0 0]
 [9 0 0 0 0 0 0]
 [8 0 0 0 0 0 0]
 [7 0 0 0 0 1 0]
 [8 0 0 0 0 0 0]]


C:\Users\jordi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\jordi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\jordi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\metrics\_classificati


CV F1-macro (mean ± std): 0.0599 ± 0.0343



## 8. Modelo 4: Parzen Windows

**Nota sobre CV:** La validación cruzada 5-fold divide el conjunto de entrenamiento en 5 partes. Para cada configuración de hiperparámetro, se entrena y evalúa 5 veces (una por fold), obteniendo 5 scores de F1-macro. El **mean** indica el rendimiento promedio y el **std** indica la estabilidad del modelo (std bajo = rendimiento consistente entre folds).

In [111]:
print("=" * 80)
print("4. DENSIDAD NO PARAMÉTRICA - PARZEN WINDOWS")
print("=" * 80)

class ParzenBayes:
    def __init__(self, bandwidth=0.5):
        self.bandwidth = bandwidth
        self.priors = None
        self.kdes = None
        self.classes = None
    
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.priors = np.bincount(y) / len(y)
        self.kdes = {}
        for c in self.classes:
            X_c = X[y == c].values.reshape(-1, X.shape[1])
            kde = KernelDensity(kernel='gaussian', bandwidth=self.bandwidth).fit(X_c)
            self.kdes[c] = kde
        return self
    
    def predict(self, X):
        X_val = X.values.reshape(-1, X.shape[1])
        n_samples = len(X_val)
        ll = np.zeros((n_samples, len(self.classes)))
        for i, c in enumerate(self.classes):
            ll[:, i] = np.exp(self.kdes[c].score_samples(X_val))
        posteriors = ll * self.priors
        posteriors /= posteriors.sum(axis=1, keepdims=True) + 1e-10
        return np.argmax(posteriors, axis=1)

print("✓ Clase ParzenBayes definida")

4. DENSIDAD NO PARAMÉTRICA - PARZEN WINDOWS
✓ Clase ParzenBayes definida


In [112]:
# GridSearch para bandwidth (h)
print("\n--- Búsqueda de hiperparámetros (en train) ---")
params_parzen = {'bandwidth': [0.05,0.1, 0.5, 1.0, 1.5, 2.0]}

best_h = None
best_cv_score = -np.inf
best_cv_scores = None  # Guardar los 5 scores del mejor

for h in params_parzen['bandwidth']:
    model_temp = ParzenBayes(bandwidth=h)
    cv_scores_temp = []
    for train_idx, val_idx in skf.split(X_train, y_train):
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]
        model_temp.fit(X_tr, y_tr)
        preds_temp = model_temp.predict(X_val)
        cv_scores_temp.append(f1_score(y_val, preds_temp, average='macro'))
    mean_score = np.mean(cv_scores_temp)
    print(f"h={h}: F1-macro CV = {mean_score:.4f}")
    if mean_score > best_cv_score:
        best_cv_score = mean_score
        best_h = h
        best_cv_scores = cv_scores_temp  # Guardar scores del mejor

print(f"\n✓ Mejor bandwidth h: {best_h}")
print(f"✓ CV F1-macro (mean ± std): {np.mean(best_cv_scores):.4f} ± {np.std(best_cv_scores):.4f}")


--- Búsqueda de hiperparámetros (en train) ---
h=0.05: F1-macro CV = 0.9302
h=0.1: F1-macro CV = 0.9350
h=0.5: F1-macro CV = 0.9388
h=1.0: F1-macro CV = 0.8729
h=1.5: F1-macro CV = 0.7364
h=2.0: F1-macro CV = 0.5948

✓ Mejor bandwidth h: 0.5
✓ CV F1-macro (mean ± std): 0.9388 ± 0.0258


In [113]:
# Entrenar con best h y evaluar
parzen_bayes = ParzenBayes(bandwidth=best_h)
parzen_bayes.fit(pd.DataFrame(X_train), y_train)
parzen_acc, parzen_f1, parzen_cm = evaluate_model(parzen_bayes, pd.DataFrame(X_test), y_test, "Parzen Bayes")


--- Resultados en Test (Parzen Bayes) ---
Accuracy: 0.9483
F1-macro: 0.9457

Reporte de clasificación:
              precision    recall  f1-score   support

      mammal       1.00      1.00      1.00         9
        bird       1.00      1.00      1.00         8
     reptile       1.00      0.88      0.93         8
        fish       0.90      1.00      0.95         9
   amphibian       0.89      1.00      0.94         8
invertebrate       0.89      1.00      0.94         8
      insect       1.00      0.75      0.86         8

    accuracy                           0.95        58
   macro avg       0.95      0.95      0.95        58
weighted avg       0.95      0.95      0.95        58


Matriz de confusión:
[[9 0 0 0 0 0 0]
 [0 8 0 0 0 0 0]
 [0 0 7 0 1 0 0]
 [0 0 0 9 0 0 0]
 [0 0 0 0 8 0 0]
 [0 0 0 0 0 8 0]
 [0 0 0 1 0 1 6]]


## 9. Modelo 5: k-NN Density Estimator

In [114]:
print("=" * 80)
print("5. DENSIDAD NO PARAMÉTRICA - k-NN ESTIMATOR")
print("=" * 80)

class KNNDensityBayes:
    def __init__(self, k=5):
        self.k = k
        self.priors = None
        self.kdes = None
        self.classes = None
    
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.priors = np.bincount(y) / len(y)
        self.kdes = {}
        for c in self.classes:
            X_c = X[y == c].values.reshape(-1, X.shape[1])
            bandwidth = 1.0 / np.sqrt(self.k / len(X_c)) if len(X_c) > 0 else 0.5
            kde = KernelDensity(kernel='gaussian', bandwidth=bandwidth, algorithm='kd_tree').fit(X_c)
            self.kdes[c] = kde
        return self
    
    def predict(self, X):
        X_val = X.values.reshape(-1, X.shape[1])
        n_samples = len(X_val)
        ll = np.zeros((n_samples, len(self.classes)))
        for i, c in enumerate(self.classes):
            ll[:, i] = np.exp(self.kdes[c].score_samples(X_val))
        posteriors = ll * self.priors
        posteriors /= posteriors.sum(axis=1, keepdims=True) + 1e-10
        return np.argmax(posteriors, axis=1)

print("✓ Clase KNNDensityBayes definida")

5. DENSIDAD NO PARAMÉTRICA - k-NN ESTIMATOR
✓ Clase KNNDensityBayes definida


In [115]:
# GridSearch para k
print("\n--- Búsqueda de hiperparámetros (en train) ---")
params_knn_density = [3, 5, 7, 9, 11]
best_k_density = None
best_cv_score_density = -np.inf
best_cv_scores_density = None  # Guardar los 5 scores del mejor

for k in params_knn_density:
    model_temp = KNNDensityBayes(k=k)
    cv_scores_temp = []
    for train_idx, val_idx in skf.split(X_train, y_train):
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]
        model_temp.fit(X_tr, y_tr)
        preds_temp = model_temp.predict(X_val)
        cv_scores_temp.append(f1_score(y_val, preds_temp, average='macro'))
    mean_score = np.mean(cv_scores_temp)
    print(f"k={k}: F1-macro CV = {mean_score:.4f}")
    if mean_score > best_cv_score_density:
        best_cv_score_density = mean_score
        best_k_density = k
        best_cv_scores_density = cv_scores_temp  # Guardar scores del mejor

print(f"\n✓ Mejor k: {best_k_density}")
print(f"✓ CV F1-macro (mean ± std): {np.mean(best_cv_scores_density):.4f} ± {np.std(best_cv_scores_density):.4f}")


--- Búsqueda de hiperparámetros (en train) ---
k=3: F1-macro CV = 0.5710
k=5: F1-macro CV = 0.5860
k=7: F1-macro CV = 0.6088
k=9: F1-macro CV = 0.6309
k=11: F1-macro CV = 0.6501

✓ Mejor k: 11
✓ CV F1-macro (mean ± std): 0.6501 ± 0.0670
k=7: F1-macro CV = 0.6088
k=9: F1-macro CV = 0.6309
k=11: F1-macro CV = 0.6501

✓ Mejor k: 11
✓ CV F1-macro (mean ± std): 0.6501 ± 0.0670


In [116]:
knn_density_bayes = KNNDensityBayes(k=best_k_density)
knn_density_bayes.fit(pd.DataFrame(X_train), y_train)
knn_d_acc, knn_d_f1, knn_d_cm = evaluate_model(knn_density_bayes, pd.DataFrame(X_test), y_test, "k-NN Density Bayes")


--- Resultados en Test (k-NN Density Bayes) ---
Accuracy: 0.7241
F1-macro: 0.6182

Reporte de clasificación:
              precision    recall  f1-score   support

      mammal       1.00      1.00      1.00         9
        bird       1.00      1.00      1.00         8
     reptile       0.00      0.00      0.00         8
        fish       0.47      1.00      0.64         9
   amphibian       0.73      1.00      0.84         8
invertebrate       0.73      1.00      0.84         8
      insect       0.00      0.00      0.00         8

    accuracy                           0.72        58
   macro avg       0.56      0.71      0.62        58
weighted avg       0.57      0.72      0.63        58


Matriz de confusión:
[[9 0 0 0 0 0 0]
 [0 8 0 0 0 0 0]
 [0 0 0 5 3 0 0]
 [0 0 0 9 0 0 0]
 [0 0 0 0 8 0 0]
 [0 0 0 0 0 8 0]
 [0 0 0 5 0 3 0]]


C:\Users\jordi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\jordi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\jordi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\metrics\_classificati

## 10. Modelo 6: k-NN Rule (Directo)

In [117]:
print("=" * 80)
print("6. K-NEAREST NEIGHBORS RULE (Directo)")
print("=" * 80)

print("\n--- Búsqueda de hiperparámetros (en train) ---")
print("Buscando mejor k con CV 5-fold...")
params_knn = {'n_neighbors': [1, 3, 5, 7, 9, 11]}
grid_knn = GridSearchCV(
    KNeighborsClassifier(metric='euclidean'),
    params_knn,
    cv=skf,
    scoring='f1_macro',
    n_jobs=-1,
    verbose=0,
    return_train_score=False
)
grid_knn.fit(X_train, y_train)

# Obtener std del mejor modelo (de los 5 folds)
best_idx = grid_knn.best_index_
best_std = grid_knn.cv_results_['std_test_score'][best_idx]

print(f"\n✓ Mejor k: {grid_knn.best_params_['n_neighbors']}")
print(f"✓ CV F1-macro (mean ± std): {grid_knn.best_score_:.4f} ± {best_std:.4f}")

6. K-NEAREST NEIGHBORS RULE (Directo)

--- Búsqueda de hiperparámetros (en train) ---
Buscando mejor k con CV 5-fold...

✓ Mejor k: 3
✓ CV F1-macro (mean ± std): 0.9359 ± 0.0338


In [118]:
best_knn = grid_knn.best_estimator_
knn_acc, knn_f1, knn_cm = evaluate_model(best_knn, X_test, y_test, "k-NN Rule")


--- Resultados en Test (k-NN Rule) ---
Accuracy: 0.9655
F1-macro: 0.9637

Reporte de clasificación:
              precision    recall  f1-score   support

      mammal       1.00      1.00      1.00         9
        bird       1.00      1.00      1.00         8
     reptile       1.00      1.00      1.00         8
        fish       0.90      1.00      0.95         9
   amphibian       1.00      1.00      1.00         8
invertebrate       0.89      1.00      0.94         8
      insect       1.00      0.75      0.86         8

    accuracy                           0.97        58
   macro avg       0.97      0.96      0.96        58
weighted avg       0.97      0.97      0.96        58


Matriz de confusión:
[[9 0 0 0 0 0 0]
 [0 8 0 0 0 0 0]
 [0 0 8 0 0 0 0]
 [0 0 0 9 0 0 0]
 [0 0 0 0 8 0 0]
 [0 0 0 0 0 8 0]
 [0 0 0 1 0 1 6]]


## 11. Comparación final: CV (Train) vs Test

La siguiente sección muestra dos comparaciones:
1. **Validación Cruzada en Train**: Rendimiento durante la selección de modelos/hiperparámetros (con std para medir estabilidad)
2. **Evaluación en Test**: Rendimiento final en datos no vistos (sin std porque solo hay 1 evaluación)

## Resumen de Validación Cruzada (5-fold en Train)

Todos los modelos fueron evaluados usando validación cruzada estratificada 5-fold **solo en el conjunto de entrenamiento**. El **mean** indica el rendimiento promedio y el **std** indica la variabilidad entre folds (menor std = mayor estabilidad).

In [119]:
print("=" * 80)
print("RESUMEN: VALIDACIÓN CRUZADA 5-FOLD (en Train)")
print("=" * 80)

cv_results = {
    'Naive Bayes': (nb_cv_scores.mean(), nb_cv_scores.std()),
    'MLE Full': (mle_cv_scores.mean(), mle_cv_scores.std()),
    'Histogram Bayes': (hist_cv_scores.mean(), hist_cv_scores.std()),
    f'Parzen (h={best_h})': (np.mean(best_cv_scores), np.std(best_cv_scores)),
    f'k-NN Density (k={best_k_density})': (np.mean(best_cv_scores_density), np.std(best_cv_scores_density)),
    f'k-NN Rule (k={grid_knn.best_params_["n_neighbors"]})': (grid_knn.best_score_, best_std)
}

print(f"\n{'Modelo':<30} {'F1-macro (mean)':>15} {'std':>10}")
print("-" * 60)
for model, (mean, std) in cv_results.items():
    print(f"{model:<30} {mean:>15.4f} {std:>10.4f}")
print("-" * 60)
print("\nInterpretación: std bajo indica mayor estabilidad del modelo entre folds")
print("=" * 80)

RESUMEN: VALIDACIÓN CRUZADA 5-FOLD (en Train)

Modelo                         F1-macro (mean)        std
------------------------------------------------------------
Naive Bayes                             0.8782     0.0354
MLE Full                                0.8889     0.0595
Histogram Bayes                         0.0599     0.0343
Parzen (h=0.5)                          0.9388     0.0258
k-NN Density (k=11)                     0.6501     0.0670
k-NN Rule (k=3)                         0.9359     0.0338
------------------------------------------------------------

Interpretación: std bajo indica mayor estabilidad del modelo entre folds

RESUMEN: VALIDACIÓN CRUZADA 5-FOLD (en Train)

Modelo                         F1-macro (mean)        std
------------------------------------------------------------
Naive Bayes                             0.8782     0.0354
MLE Full                                0.8889     0.0595
Histogram Bayes                         0.0599     0.0343
Parzen (h=

In [120]:
print("=" * 80)
print("COMPARACIÓN FINAL DE MODELOS (en Test)")
print("=" * 80)

results = {
    'Naive Bayes': (nb_acc, nb_f1),
    'MLE Full': (mle_acc, mle_f1),
    'Histogram Bayes': (hist_acc, hist_f1),
    'Parzen Bayes': (parzen_acc, parzen_f1),
    'k-NN Density Bayes': (knn_d_acc, knn_d_f1),
    f'k-NN Rule (k={grid_knn.best_params_["n_neighbors"]})': (knn_acc, knn_f1)
}

print(f"\n{'Modelo':<25} {'Accuracy':>10} {'F1-macro':>10}")
print("-" * 50)
for model, (acc, f1) in results.items():
    print(f"{model:<25} {acc:>10.4f} {f1:>10.4f}")
print("-" * 50)

# Mejor modelo por F1-macro (prioridad para multiclass)
best_model = max(results, key=lambda k: results[k][1])
print(f"\n✓ Mejor modelo (por F1-macro): {best_model} (F1: {results[best_model][1]:.4f})")

print("\n" + "=" * 80)
print("✓ Análisis completo finalizado")
print("=" * 80)

COMPARACIÓN FINAL DE MODELOS (en Test)

Modelo                      Accuracy   F1-macro
--------------------------------------------------
Naive Bayes                   0.8966     0.8885
MLE Full                      0.9310     0.9281
Histogram Bayes               0.1724     0.0707
Parzen Bayes                  0.9483     0.9457
k-NN Density Bayes            0.7241     0.6182
k-NN Rule (k=3)               0.9655     0.9637
--------------------------------------------------

✓ Mejor modelo (por F1-macro): k-NN Rule (k=3) (F1: 0.9637)

✓ Análisis completo finalizado
